In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils

sys.setrecursionlimit(40000)

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import cv2
from keras_frcnn.mylayer import mylayer

In [3]:
K.set_learning_phase(1) #set learning phase

In [4]:
parser = OptionParser()

parser.add_option("-p", "--path", dest="train_path", help="Path to training data.",default = 'data.txt')
parser.add_option("-o", "--parser", dest="parser", help="Parser to use. One of simple or pascal_voc",
				default="simple")
parser.add_option("-n", "--num_rois", type="int", dest="num_rois", help="Number of RoIs to process at once.", default=32) #32
parser.add_option("--network", dest="network", help="Base network to use. Supports vgg or resnet50.", default='resnet50')
parser.add_option("--hf", dest="horizontal_flips", help="Augment with horizontal flips in training. (Default=false).", action="store_true", default=False)
parser.add_option("--vf", dest="vertical_flips", help="Augment with vertical flips in training. (Default=false).", action="store_true", default=False)
parser.add_option("--rot", "--rot_90", dest="rot_90", help="Augment with 90 degree rotations in training. (Default=false).",
				  action="store_true", default=False)
parser.add_option("--num_epochs", type="int", dest="num_epochs", help="Number of epochs.", default=3) #2000
parser.add_option("--config_filename", dest="config_filename", help=
				"Location to store all the metadata related to the training (to be used when testing).",
				default="config.pickle")
parser.add_option("--output_weight_path", dest="output_weight_path", help="Output path for weights.", default='./model_frcnn.hdf5')
parser.add_option("--input_weight_path", dest="input_weight_path", help="Input path for weights. If not specified, will try to load default weights provided by keras.")

#(options, args) = parser.parse_args()

<Option at 0xb2bd15710: --input_weight_path>

In [5]:
train_path ='data.txt'
parser = 'simple'
num_rois = 3
network ='vgg'
horizontal_flips = False
vertical_flips = False
rot_90 = False
num_epochs = 3
config_filename = 'config.pickle'
output_weight_path = './model_frcnn.hdf5'
#input_weight_path = 

In [6]:
if not train_path:   # if filename is not given
	parser.error('Error: path to training data must be specified. Pass --path to command line')

if parser == 'pascal_voc':
	from keras_frcnn.pascal_voc_parser import get_data
elif parser == 'simple':
	from keras_frcnn.simple_parser import get_data
else:
	raise ValueError("Command line option parser must be one of 'pascal_voc' or 'simple'")

# pass the settings from the command line, and persist them in the config object
C = config.Config()

C.use_horizontal_flips = bool(horizontal_flips)
C.use_vertical_flips = bool(vertical_flips)
C.rot_90 = bool(rot_90)

C.model_path = output_weight_path
C.num_rois = int(num_rois)

if network == 'vgg':
	C.network = 'vgg'
	from keras_frcnn import vgg as nn
elif network == 'resnet50':
	from keras_frcnn import resnet as nn
	C.network = 'resnet50'
else:
	print('Not a valid model')
	raise ValueError

In [7]:
# check if weight path was passed via command line
# if input_weight_path:
# 	C.base_net_weights = options.input_weight_path
# else:
# 	# set the path to weights based on backend and model
# 	C.base_net_weights = nn.get_weight_path()

C.base_net_weights = nn.get_weight_path()
all_imgs, classes_count, class_mapping = get_data(train_path)

IOError: [Errno 2] No such file or directory: 'data.txt'

In [47]:

if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))

config_output_filename = config_filename

with open(config_output_filename, 'wb') as config_f:
	pickle.dump(C,config_f)
	print('Config has been written to {}, and can be loaded when testing to ensure correct results'.format(config_output_filename))

random.shuffle(all_imgs)

num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))



Training images per class:
{'bg': 0, 'dress': 30}
Num classes (including bg) = 2
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 25
Num val samples 5


In [48]:
data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.image_dim_ordering(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.image_dim_ordering(), mode='val')

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (400, 400, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(6, 4))

#pdb.set_trace()
# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers, num_anchors)

In [49]:
# cv2.imwrite('a.jpg',next(data_gen_train)[0][0,:,:,:])
# next(data_gen_train)[0][0,:,:,:].shape

In [50]:
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
def temp_classifier(base_layers, input_rois, num_rois, nb_classes = 21, trainable=False):

    # compile times on theano tend to be very high, so we use smaller ROI pooling regions to workaround

    if K.backend() == 'tensorflow':
        pooling_regions = 7
        input_shape = (num_rois,7,7,512)
    elif K.backend() == 'theano':
        pooling_regions = 7
        input_shape = (num_rois,512,7,7)

    #out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([base_layers, input_rois])
    out_roi_pool = RoiPoolingConv(3, num_rois)([base_layers, input_rois])
    print out_roi_pool

    out = TimeDistributed(Flatten(name='flatten'))(out_roi_pool)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc1'))(out)
    out = TimeDistributed(Dropout(0.5))(out)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc2'))(out)
    out = TimeDistributed(Dropout(0.5))(out)

    out_class = TimeDistributed(Dense(nb_classes, activation='softmax', kernel_initializer='zero'))(out)
    # note: no regression target for bg class
    #out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)

    return out_class

def temp_classifier1(base_layers, input_rois, num_rois, nb_classes = 21, trainable=False):

    # compile times on theano tend to be very high, so we use smaller ROI pooling regions to workaround

    if K.backend() == 'tensorflow':
        pooling_regions = 7
        input_shape = (num_rois,7,7,512)
    elif K.backend() == 'theano':
        pooling_regions = 7
        input_shape = (num_rois,512,7,7)

    out_roi_pool = mylayer(pooling_regions, num_rois)([base_layers, input_rois])
    print out_roi_pool

    out = (Flatten(name='flatten'))(out_roi_pool)
    out = (Dense(4096, activation='relu', name='fc1'))(out)
    out = (Dropout(0.5))(out)
    #out = (Dense(4096, activation='relu', name='fc2'))(out)
    #out = (Dropout(0.5))(out)
    print 'no of classes', nb_classes
    out_class = (Dense(nb_classes, activation='sigmoid', kernel_initializer='zero'))(out)
    # note: no regression target for bg class
    #out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)

    return out_class


In [51]:
#classifier = temp_classifier1(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=False)
classifier = temp_classifier1(shared_layers, roi_input, 6, nb_classes=564, trainable=False)



#model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
#model_all = Model([img_input, roi_input], rpn[:2] + classifier)

try:
	print('loading weights from {}'.format(C.base_net_weights))
	#model_rpn.load_weights(C.base_net_weights, by_name=True)
	#model_classifier.load_weights(C.base_net_weights, by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

Tensor("mylayer_2/Reshape:0", shape=(1, 6, 7, 7, 512), dtype=float32)
no of classes 564
loading weights from vgg16_weights_tf_dim_ordering_tf_kernels.h5


In [52]:
optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_classifier.compile(optimizer=optimizer_classifier, loss = 'binary_crossentropy')
#output_layers = ['block5_conv3','roi_pooling_conv_4']
#model_classifier.metrics_tensors += [layer.output for layer in model_classifier.layers if layer.name in output_layers]

In [53]:
import pandas as pd
df = pd.read_pickle('rois_new.pkl')

In [65]:
df.head()

,Unnamed: 0,x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6,w,h,img_name,atts,landmarks
0,0,74,101,22,133,64.0,105.0,23,16,163,176,150.0,153.0,161,195,img_n/Sheer_Pleated-Front_Blouse/img_00000001.jpg,[717 818],"[[153, 17, 60, 60], [220, 2, 60, 60], [24, 304..."
1,1,40,55,12,73,39.0,67.0,8,8,19,32,91.0,93.0,89,103,img_n/Sheer_Pleated-Front_Blouse/img_00000002.jpg,[717 818],"[[149, 1, 60, 60], [217, 1, 60, 60], [23, 43, ..."
2,2,36,51,13,76,22.0,53.0,14,11,16,19,123.0,120.0,92,136,img_n/Sheer_Pleated-Front_Blouse/img_00000003.jpg,[141 717 837 956],"[[126, 11, 60, 60], [191, 2, 60, 60], [26, 17,..."
3,3,45,62,16,97,49.0,69.0,12,10,109,109,86.0,86.0,117,121,img_n/Sheer_Pleated-Front_Blouse/img_00000004.jpg,[716],"[[123, 9, 60, 60], [181, 3, 60, 60], [24, 330,..."
4,4,56,70,17,100,59.0,91.0,18,14,106,128,157.0,158.0,121,175,img_n/Sheer_Pleated-Front_Blouse/img_00000005.jpg,[349 405 717 810],"[[155, 11, 60, 60], [201, 2, 60, 60], [26, 212..."


In [16]:
df = df.sample(frac=1).reset_index(drop=True).drop(columns = ['atts'])

In [17]:
df2 = pd.read_pickle('../processed_data/list_att_img.pkl').rename(columns = {'name':'img_name'})

In [18]:
def imgname_change(row):
    a = str.split(row,'/')
    return a[0] + '_n/' + a[1] + '/' + a[2]

In [19]:
df2['img_name'] = df2['img_name'].apply(lambda x: imgname_change(x))

In [20]:
df3 = df.merge(df2, on = 'img_name', how = 'inner')

In [21]:
df3.head()

,Unnamed: 0,x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6,w,h,img_name,landmarks,atts
0,113253,33,73,17,104,37.0,75.0,23,14,90,80,152.0,154.0,126,166,img_n/Longline_Baseball_Jersey/img_00000026.jpg,"[[74, 25, 134, 85], [201, 3, 261, 63], [23, 18...",[571]
1,85984,55,109,22,130,43.0,116.0,19,15,57,57,165.0,165.0,157,183,img_n/Elephant_Print_Tee/img_00000057.jpg,"[[110, 11, 170, 71], [247, 2, 307, 62], [26, 9...","[307, 308, 730, 883]"
2,87879,28,81,20,118,28.0,87.0,23,16,108,176,179.0,179.0,143,191,img_n/Eyelash-Knit_Batwing_Cardigan/img_000000...,"[[48, 18, 108, 78], [196, 3, 256, 63], [25, 19...",[560]
3,59136,72,121,25,149,56.0,113.0,15,18,79,88,160.0,164.0,180,181,img_n/Yes_Please_Paneled_Tee/img_00000012.jpg,"[[130, 3, 190, 63], [238, 9, 298, 69], [25, 14...","[44, 119, 715]"
4,135998,40,110,21,128,38.0,108.0,10,10,89,98,110.0,108.0,155,122,img_n/Boxy_Distressed_Tee/img_00000006.jpg,"[[73, 2, 133, 62], [253, 2, 313, 62], [24, 261...","[93, 268, 394, 512, 577, 705, 867]"


In [22]:
img = []
landmarks = []
attribs = []
for i in range(1000):
    print i
    a = cv2.imread('../' + df3.loc[i,'img_name'])
    a = cv2.resize(a,(400,400))
    img.append(a)
    lnd = df3.loc[i,'landmarks']
    landmarks.append(lnd)
    atts = df3.loc[i,'atts']
    attribs.append(atts)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
# mlb.fit_transform([(1, 2), (3,)])
# array([[1, 1, 0],
#        [0, 0, 1]])
# >>> mlb.classes_
# array([1, 2, 3])

In [24]:
X = np.array(img)
X2 = np.array(landmarks)
Y = mlb.fit_transform(attribs)

In [25]:
X.shape, X2.shape, Y.shape

((1000, 400, 400, 3), (1000, 6, 4), (1000, 557))

In [26]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
checkpoint = ModelCheckpoint("att_prediction.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)


In [27]:
# out = model_classifier.fit([X[0:1,:,:,:],X2[0:1,:,:]],Y[0:1,:],batch_size=1,epochs=1,verbose=1,shuffle=False)

In [28]:

data = np.load('mat.npz')
Xa =  data['name1']
Xb = data['name2']
Ya = data['name3']
#print data['name2']

In [29]:
Ya = np.random.randint(1,size = (2,564))

In [30]:
X2[:,:,2:] =  np.ones((1000,6,2)) * 30

In [31]:
X2[:,:,0:2] =  np.ones((1000,6,2)) * 20

In [32]:
X2[0:1,0:6,:], Xb[0:1,0:6,:]

(array([[[20, 20, 30, 30],
         [20, 20, 30, 30],
         [20, 20, 30, 30],
         [20, 20, 30, 30],
         [20, 20, 30, 30],
         [20, 20, 30, 30]]]), array([[[ 7, 20, 23, 25],
         [11, 19, 23, 26],
         [ 3, 17, 23, 28],
         [12, 10, 23, 35],
         [10,  7, 26, 32],
         [ 8, 11, 23, 34]]]))

In [35]:
#a = model_classifier.train_on_batch([X[:,0:350,0:350,:], X2[:,0:4,:]], Y1[:,0:4,:])
a = model_classifier.train_on_batch([X[0:1,0:400,0:400,:], Xb[0:1,0:6,:] ], Ya[0:1,:])
a
#[Y1[:,sel_samples,:], Y2[:,sel_samples,:]]
#[X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, 0:4]]
# 600 and 500 worked 350, 350

Instructions for updating:
Use tf.cast instead.


0.69314694

In [87]:
img = X
rois = X2[:,0:3,:]
x = rois[0, roi_idx, 0]
y = rois[0, roi_idx, 1]
w = rois[0, roi_idx, 2]
h = rois[0, roi_idx, 3]

row_length = w / float(7)
col_length = h / float(7)

num_pool_regions = 7
x = K.cast(x, 'int32')
y = K.cast(y, 'int32')
w = K.cast(w, 'int32')
h = K.cast(h, 'int32')
[x,y,w,h] =tf.Session().run([x,y,w,h])
rs = tf.image.resize_images(img[:, y:y+h, x:x+w, :], (50,50))

In [85]:
[x,y,w,h] =tf.Session().run([x,y,w,h])

In [22]:
#assert(len(x) == 2)
import pdb
import tensorflow as tf
img = X[:,0:300,0:350,:]
rois = X2[:,0:3,:]

input_shape = K.shape(img)
pdb.set_trace()
outputs = []

for roi_idx in range(3):

    x = rois[0, roi_idx, 0]
    y = rois[0, roi_idx, 1]
    w = rois[0, roi_idx, 2]
    h = rois[0, roi_idx, 3]

    row_length = w / float(7)
    col_length = h / float(7)

    num_pool_regions = 7
    x = K.cast(x, 'int32')
    y = K.cast(y, 'int32')
    w = K.cast(w, 'int32')
    h = K.cast(h, 'int32')
    #[x,y,w,h] =tf.Session().run([x,y,w,h])
    print x,y,w,h
    rs = tf.image.resize_images(img[0, y:y+h, x:x+w, :], (7,7))
    outputs.append(rs)

final_output = K.concatenate(outputs, axis=0)
final_output = K.reshape(final_output, (1, 3, 7, 7, 3))
#pdb.set_trace()
final_output = K.permute_dimensions(final_output, (0, 1, 2, 3, 4))
print final_output.shape


--Return--
> <ipython-input-22-c4220a04666e>(8)<module>()->None
-> pdb.set_trace()
(Pdb) c
Tensor("Cast_24:0", shape=(), dtype=int32) Tensor("Cast_25:0", shape=(), dtype=int32) Tensor("Cast_26:0", shape=(), dtype=int32) Tensor("Cast_27:0", shape=(), dtype=int32)


TypeError: slice indices must be integers or None or have an __index__ method

In [29]:
final_output.shape

TensorShape([Dimension(1), Dimension(3), Dimension(7), Dimension(7), Dimension(3)])

In [23]:
q = tf.image.resize_images(X[0,0:350,0:340,:],(7,7))
K.reshape(q,(1,7,7,3))

<tf.Tensor 'Reshape_2:0' shape=(1, 7, 7, 3) dtype=float32>

In [121]:
x = X[:,0:350,0:350,:]
y = X2[:,0:4,:]
z = Y1[:,0:4,:]

In [144]:
z = np.append(z,z,axis =0)

In [154]:
z1 = np.random.randint(1,size = (16,2))

In [48]:
model_classifier.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 400, 400, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 400, 400, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 200, 200, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con